# Global Lesioning Analysis

Note: The first time this is run (i.e., with an empty cache), it will
take 2-3 hours from beginning to end.

In [1]:
import textwrap
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from prep_data import \
    mazes,\
    model_preds,\
    get_exp1_at,\
    get_exp2_at,\
    get_exp3_at__exp3_mt,\
    get_exp4a_ht,\
    get_exp4b_ht
from analysisutils import short_predictor_names
import analysisutils

from vgc_project.r import create_R_model_interface, ImmutableDataFrame
joblib_cache_location = "./_analysiscache"
rmods = create_R_model_interface(joblib_cache_location=joblib_cache_location)


# Dynamic

In [2]:
predictors = [
    'dynamic_vgc_weight_Z',
    'log_traj_based_hitcount_Z',
    'graph_based_hitcount_Z',
    'bottleneck_dist_Z',
    'sr_occ_Z',
    'nav_mindist_Z',
    'nav_mindist_timestep_Z',
    'optpolicy_dist_Z',
    'goal_dist_Z',
    'start_dist_Z',
    'walls_dist_Z',
    'center_dist_Z',
]
exp_analysis_predictors = dict(
    exp1=[p for p in predictors if "optpolicy_" not in p],
    exp2_early=[p for p in predictors if "nav_mindist" not in p],
    exp2_full=[p for p in predictors if "optpolicy_" not in p],
    exp3=[p for p in predictors if "optpolicy_" not in p],
    exp4=[p for p in predictors if "nav_mindist" not in p],
)
    


In [3]:
exp1_at = get_exp1_at()
exp1_at_im = ImmutableDataFrame(exp1_at)
exp1_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp1']),
    data=exp1_at_im
)
exp1_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 1 Accuracy",
    data=exp1_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp1'],
    rmods=rmods
)

exp2_at = get_exp2_at()
exp2_early_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'earlyterm'])
exp2_full_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'full'])
exp2_at_im = ImmutableDataFrame(exp2_at)
exp2_full_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp2_full']),
    data=exp2_full_at_im
)
exp2_early_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp2_early']),
    data=exp2_early_at_im
)
exp2_early_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 2 Accuracy Early",
    data=exp2_early_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp2_early'],
    rmods=rmods
)
exp2_full_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 2 Accuracy Full",
    data=exp2_full_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp2_full'],
    rmods=rmods
)


exp3_at, exp3_mt = get_exp3_at__exp3_mt()
exp3_at_im = ImmutableDataFrame(exp3_at)
exp3_mt_im = ImmutableDataFrame(exp3_mt)
exp3_correct_global_model = rmods.lmer(
    formula="correct ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_mt_im
)
exp3_conf_global_model = rmods.lmer(
    formula="conf_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_mt_im
)
exp3_attn_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_at_im
)
exp3_correct_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Accuracy",
    data=exp3_mt_im,
    dv="correct",
    model_func="glmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)
exp3_conf_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Confidence",
    data=exp3_mt_im,
    dv="conf_N",
    model_func="lmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)
exp3_attn_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Awareness",
    data=exp3_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)


exp4a_ht = get_exp4a_ht()
exp4b_ht = get_exp4b_ht()
exp4a_ht_im = ImmutableDataFrame(exp4a_ht)
exp4b_ht_im = ImmutableDataFrame(exp4b_ht)
exp4a_hov_global_model = rmods.glmer(
    formula="hovered ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4a_ht_im
)
exp4a_dur_global_model = rmods.lmer(
    formula="log_hoverduration ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4a_ht_im
)
exp4b_hov_global_model = rmods.glmer(
    formula="hovered ~ (1 | sessionId) + (1 | grid) +"+"+".join(exp_analysis_predictors['exp4']),
    data=exp4b_ht_im
)
exp4b_dur_global_model = rmods.lmer(
    formula="log_hoverduration ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4b_ht_im
)
exp4a_hov_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4a Hover",
    data=exp4a_ht_im,
    dv="hovered",
    model_func="glmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4a_dur_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4a Hover Duration",
    data=exp4a_ht_im,
    dv="log_hoverduration",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4b_hov_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4b Hover",
    data=exp4b_ht_im,
    dv="hovered",
    model_func="glmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4b_dur_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4b Hover Duration",
    data=exp4b_ht_im,
    dv="log_hoverduration",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)

Loading Experiment 1 Attention Trials


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:02<00:00,  4.54it/s]


## Tables with all estimated coefficients

In [4]:
def format_coefficients(mod):
    return pd.DataFrame(mod['coefficients']).\
        set_index('effect').\
        rename(columns={"estimate": r"$\beta$", "se": "SE"})
all_global_models = [
    dict(model=exp1_global_model, name=("Initial Exp", "Awareness")),
    dict(model=exp2_full_global_model, name=("Up-front Exp", "Awareness (F)")),
    dict(model=exp2_early_global_model, name=("Up-front Exp","Awareness (E)")),
    dict(model=exp3_correct_global_model, name=("Critical Mazes Exp", "Accuracy")),
    dict(model=exp3_conf_global_model, name=("Critical Mazes Exp", "Confidence")),
    dict(model=exp3_attn_global_model, name=("Critical Mazes Exp", "Awareness")),
    dict(model=exp4a_hov_global_model, name=("Process-Tracing Exp (Initial Mazes)", "Hovering")),
    dict(model=exp4a_dur_global_model, name=("Process-Tracing Exp (Initial Mazes)", "Duration")),
    dict(model=exp4b_hov_global_model, name=("Process-Tracing Exp (Critical Mazes)", "Hovering")),
    dict(model=exp4b_dur_global_model, name=("Process-Tracing Exp (Critical Mazes)", "Duration")),
]
coeff_table = pd.concat(
    [format_coefficients(mod['model']) for mod in all_global_models],
    axis=1,
    keys=[mod['name'] for mod in all_global_models]
)
coeff_table = coeff_table.reindex(["(Intercept)"]+predictors)
coeff_table.index = ["Intercept" if "Intercept" in i else short_predictor_names[i.replace("_Z", "")] for i in coeff_table.index.values]

In [5]:
coeff_table1_3 = coeff_table[["Initial Exp", "Up-front Exp", "Critical Mazes Exp"]]
coeff_string1_3 = coeff_table1_3.round(3).to_latex(
    na_rep="---", 
    escape=False,
    multicolumn_format='c',
    column_format="lrrrrrrrrrrrr",
    col_space=1,
)
coeff_string1_3 = coeff_string1_3.split('\n')

coeff_string1_3.insert(
    4, r"\cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}"
)
coeff_string1_3.insert(
    3, r"\cmidrule(lr){2-3} \cmidrule(lr){5-6} \cmidrule(lr){9-12}"
)
coeff_string1_3 = "\n".join(coeff_string1_3)
with open("results/coeff_table1_3.tex", "w") as file:
    file.write(
        textwrap.dedent(r"""
        \documentclass{standalone}
        \usepackage{booktabs}
        \usepackage{tabularx}
        \usepackage{caption}
        \usepackage[scaled=0.7025996185887785]{helvet}
        \linespread{0.7025996185887785}
        \renewcommand{\familydefault}{\sfdefault}
        \begin{document}
        \fontsize{7}{8.4}
        """) + \
        coeff_string1_3 + \
        textwrap.dedent(r"""
        \end{document}
        """)
    )
# with open("results/coeff_table1_3.tex", "w") as file:
#     file.write(
#         textwrap.dedent(r"""
#         \documentclass{standalone}
#         \usepackage{booktabs}
#         \usepackage{tabularx}
#         \renewcommand{\familydefault}{\sfdefault}
#         \begin{document}
#         \fontsize{7}{8.4}
#         \selectfont
#         """) + \
#         coeff_string1_3 + \
#         textwrap.dedent(r"""
#         \end{document}
#         """)
#     )

# Create pdf to combine in vector image editor
# process = subprocess.Popen("pdflatex -output-directory results results/coeff_table1_3.tex".split())
process = subprocess.Popen("xelatex -output-directory results results/coeff_table1_3.tex".split())
output, error = process.communicate()
print(output, error)
coeff_table1_3

This is XeTeX, Version 3.14159265-2.6-0.99992 (TeX Live 2015) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./results/coeff_table1_3.tex
LaTeX2e <2015/01/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2012/09/15 v1.1b Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifluatex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/ifxetex/ifxetex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2015/texmf

Initial Exp            Up-front Exp                          \
                Awareness           Awareness (F)           Awareness (E)   
                  $\beta$        SE       $\beta$        SE       $\beta$   
Intercept        0.529834  0.016796      0.324779  0.018954      0.493559   
VGC              0.085453  0.003773      0.095453  0.005609      0.080216   
Traj HS          0.000406  0.005055      0.002636  0.007571      0.015281   
Graph HS         0.028406  0.004962      0.012309  0.007345      0.005318   
Bottleneck      -0.028425  0.003350     -0.044132  0.005039     -0.051553   
SR Overlap      -0.030582  0.003101     -0.035543  0.004604     -0.037075   
Nav Dist        -0.114215  0.003400     -0.276905  0.012018           NaN   
Nav Dist Step    0.014445  0.003829     -0.027348  0.003462           NaN   
Opt Dist              NaN       NaN           NaN       NaN     -0.084998   
Goal Dist       -0.006653  0.005075      0.004414  0.007333     -0.006317   
Start Dist      -0.020855  0.004707      0.016499  0.006932     -0.022777   
Wall Dist       -0.237352  0.024136     -0.153225  0.037296     -0.188682   
Center Dist      0.237922  0.024666      0.182157  0.038121      0.211851   

                        Critical Mazes Exp                                 \
                                  Accuracy           Confidence             
                     SE            $\beta$        SE    $\beta$        SE   
Intercept      0.015759           0.558726  0.065780   0.642280  0.041225   
VGC            0.005570           0.150333  0.014720   0.118604  0.008652   
Traj HS        0.007743           0.025193  0.029617  -0.011850  0.017408   
Graph HS       0.007454          -0.032333  0.034337  -0.034989  0.020183   
Bottleneck     0.005158           0.004206  0.013749  -0.012510  0.008083   
SR Overlap     0.004750          -0.029825  0.045753   0.006287  0.026936   
Nav Dist            NaN          -0.034225  0.012016  -0.008532  0.007075   
Nav Dist Step       NaN          -0.008778  0.010479  -0.007478  0.006264   
Opt Dist       0.005310                NaN       NaN        NaN       NaN   
Goal Dist      0.007437           0.118519  0.072951  -0.092801  0.042899   
Start Dist     0.006979           0.070526  0.068694  -0.124550  0.040391   
Wall Dist      0.036791          -0.036255  0.068543  -0.091808  0.040299   
Center Dist    0.037582           0.017856  0.057683   0.112779  0.033905   

                                   
              Awareness            
                $\beta$        SE  
Intercept      0.426516  0.043834  
VGC            0.212605  0.009222  
Traj HS        0.012026  0.018478  
Graph HS      -0.039888  0.021346  
Bottleneck    -0.033942  0.008542  
SR Overlap     0.090612  0.028434  
Nav Dist      -0.063092  0.007363  
Nav Dist Step -0.042957  0.006567  
Opt Dist            NaN       NaN  
Goal Dist      0.086154  0.045652  
Start Dist    -0.012347  0.042972  
Wall Dist      0.071401  0.042948  
Center Dist    0.004691  0.036201

In [6]:
!xelatex -output-directory results results/coeff_table1_3.tex

This is XeTeX, Version 3.14159265-2.6-0.99992 (TeX Live 2015) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./results/coeff_table1_3.tex
LaTeX2e <2015/01/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2012/09/15 v1.1b Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifluatex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/ifxetex/ifxetex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2015/texmf

In [7]:
coeff_table4 = coeff_table[["Process-Tracing Exp (Initial Mazes)", "Process-Tracing Exp (Critical Mazes)"]].dropna(axis=0)
coeff_string4 = coeff_table4.round(3).to_latex(
        na_rep="---", 
        escape=False,
        multicolumn_format='c',
        column_format="lrrrrrrrrrrrr",
        col_space=1,
    )
coeff_string4 = coeff_string4.split('\n')

coeff_string4.insert(
    4, r"\cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9}"
)
coeff_string4.insert(
    3, r"\cmidrule(lr){2-5} \cmidrule(lr){6-9}"
)
coeff_string4 = "\n".join(coeff_string4)
# with open("results/coeff_table4.tex", "w") as file:
#     file.write(
#         textwrap.dedent(r"""
#         \documentclass{standalone}
#         \usepackage{booktabs}
#         \usepackage{tabularx}
#         \renewcommand{\familydefault}{\sfdefault}
#         \begin{document}
#         \fontsize{7}{8.4}
#         \selectfont
#         """) + \
#         coeff_string4 + \
#         textwrap.dedent(r"""
#         \end{document}
#         """)
#     )
with open("results/coeff_table4.tex", "w") as file:
    file.write(
        textwrap.dedent(r"""
        \documentclass{standalone}
        \usepackage{booktabs}
        \usepackage{tabularx}
        \usepackage{caption}
        \usepackage[scaled=0.7025996185887785]{helvet}
        \linespread{0.7025996185887785}
        \renewcommand{\familydefault}{\sfdefault}
        \begin{document}
        \fontsize{7}{8.4}
        """) + \
        coeff_string4 + \
        textwrap.dedent(r"""
        \end{document}
        """)
    )
    
# Create pdf to combine in vector image editor
# process = subprocess.Popen("pdflatex -output-directory results results/coeff_table4.tex".split())
process = subprocess.Popen("xelatex -output-directory results results/coeff_table4.tex".split())
output, error = process.communicate()
print(output, error)
coeff_table4

This is XeTeX, Version 3.14159265-2.6-0.99992 (TeX Live 2015) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./results/coeff_table4.tex
LaTeX2e <2015/01/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2012/09/15 v1.1b Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifluatex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/ifxetex/ifxetex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2015/texmf-d

Process-Tracing Exp (Initial Mazes)                                \
                                       Hovering            Duration             
                                        $\beta$        SE   $\beta$        SE   
Intercept                              0.715294  0.128513  6.514217  0.071164   
VGC                                    0.590300  0.035067  0.234237  0.017443   
Traj HS                                0.459567  0.046010  0.104488  0.028091   
Graph HS                              -0.200793  0.044554 -0.091362  0.023817   
Bottleneck                            -0.219820  0.031080 -0.174067  0.016201   
SR Overlap                            -0.017689  0.028050  0.023840  0.015559   
Opt Dist                              -0.410489  0.031514 -0.025857  0.019600   
Goal Dist                             -0.146977  0.044747  0.025750  0.023741   
Start Dist                            -0.330249  0.042459  0.047930  0.022405   
Wall Dist                             -0.970381  0.207023 -0.905279  0.116937   
Center Dist                            1.278999  0.211429  1.140385  0.119106   

            Process-Tracing Exp (Critical Mazes)                                
                                        Hovering            Duration            
                                         $\beta$        SE   $\beta$        SE  
Intercept                               0.511695  0.318787  5.860651  0.181719  
VGC                                     0.711184  0.054220  0.484752  0.030048  
Traj HS                                 0.196132  0.134043  0.345269  0.079519  
Graph HS                                0.403685  0.140462 -0.296622  0.083614  
Bottleneck                              0.058083  0.059311 -0.058277  0.030562  
SR Overlap                              1.373943  0.220067  0.408484  0.105567  
Opt Dist                                0.332864  0.053661  0.152784  0.031693  
Goal Dist                              -0.992732  0.332875  0.099814  0.189575  
Start Dist                             -1.582516  0.310141 -0.168578  0.182351  
Wall Dist                              -0.110587  0.252307 -0.316952  0.148154  
Center Dist                             1.259638  0.189346  0.807273  0.127729

## Create lesioned table

In [8]:
all_lesioning_res = [
    dict(res=exp1_lesioning_res, name=("Initial Exp", "Awareness")),
    dict(res=exp2_full_lesioning_res, name=("Up-front Exp", "Awareness (F)")),
    dict(res=exp2_early_lesioning_res, name=("Up-front Exp","Awareness (E)")),
    dict(res=exp3_correct_lesioning_res, name=("Critical Mazes Exp", "Accuracy")),
    dict(res=exp3_conf_lesioning_res, name=("Critical Mazes Exp", "Confidence")),
    dict(res=exp3_attn_lesioning_res, name=("Critical Mazes Exp", "Awareness")),
    dict(res=exp4a_hov_lesioning_res, name=("Process-Tracing Exp (Initial Mazes)", "Hovering")),
    dict(res=exp4a_dur_lesioning_res, name=("Process-Tracing Exp (Initial Mazes)", "Duration")),
    dict(res=exp4b_hov_lesioning_res, name=("Process-Tracing Exp (Critical Mazes)", "Hovering")),
    dict(res=exp4b_dur_lesioning_res, name=("Process-Tracing Exp (Critical Mazes)", "Duration")),
]

### Likelihood ratio test summary

In [9]:
all_summaries = []
for mod in all_lesioning_res:
    all_summaries.append(
        f"{', '.join(mod['name'])}: {mod['res'].likelihood_ratio_test('dynamic_vgc_weight_Z', 1)}"
    )
all_summaries = '; \n'.join(all_summaries)
with open("./inputs/all_plan_exp_vgc_global_lesion_llr_dvgc.tex", "w") as file:
    file.write(all_summaries)
print(all_summaries)

Initial Exp, Awareness: $\chi^2(1) = 502.33, p < 1.0 \times 10^{-16}$; 
Up-front Exp, Awareness (F): $\chi^2(1) = 282.54, p < 1.0 \times 10^{-16}$; 
Up-front Exp, Awareness (E): $\chi^2(1) = 203.27, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Accuracy: $\chi^2(1) = 114.98, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Confidence: $\chi^2(1) = 182.26, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Awareness: $\chi^2(1) = 489.35, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Initial Mazes), Hovering: $\chi^2(1) = 295.95, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Initial Mazes), Duration: $\chi^2(1) = 178.34, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Critical Mazes), Hovering: $\chi^2(1) = 186.63, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Critical Mazes), Duration: $\chi^2(1) = 251.32, p < 1.0 \times 10^{-16}$


In [10]:
def format_global_lesioning_res(res):
    return \
        res[['dAIC.FULL', 'lesioned_effect']].\
        set_index('lesioned_effect')["dAIC.FULL"].drop('FULL')

global_lesion_table = pd.concat(
    [format_global_lesioning_res(res['res']) for res in all_lesioning_res],
    axis=1,
    keys=[res['name'] for res in all_lesioning_res]
)


global_lesion_table = global_lesion_table.reindex(predictors)
global_lesion_table.index = ["Intercept" if "Intercept" in i else short_predictor_names[i.replace("_Z", "")] for i in global_lesion_table.index.values]

# with annotation
col_ranks = global_lesion_table.rank(axis=0, ascending=False)
annot_global_lesion_table = []
for model, row in global_lesion_table.iterrows():
    annot_row = []
    for measure, daic in row.to_dict().items():
        if np.isnan(daic):
            annot_row.append("---")
            continue
            
        daic_rank = col_ranks.loc[model, measure]
        if daic_rank == 1:
            annot = "$^*$"
        elif daic_rank == 2:
            annot = r"\textsuperscript{\textdagger}"
        else:
            annot = ""
        annot_row.append(str(int(daic))+annot)
    annot_global_lesion_table.append(annot_row)

annot_global_lesion_table = pd.DataFrame(annot_global_lesion_table, index=global_lesion_table.index, columns=global_lesion_table.columns)
global_lesion_string = annot_global_lesion_table.to_latex(
    column_format="lcccccccccc",
    col_space=1,
    escape=False,
    # caption=
)

# without annotation
# global_lesion_string = global_lesion_table.round(0).to_latex(
#     na_rep="---", 
#     float_format=lambda f: str(int(f)) if not np.isnan(f) else "---",
#     column_format="lcccccccccc",
#     col_space=1,
#     escape=False
# )


global_lesion_string = global_lesion_string.replace(
    r"{} &                       Initial Exp & \multicolumn{2}{l}{Up-front Exp} & \multicolumn{3}{l}{Critical Mazes Exp} & \multicolumn{2}{l}{Process-Tracing Exp (Initial Mazes)} & \multicolumn{2}{l}{Process-Tracing Exp (Critical Mazes)} \\",
    r"{} & Initial Exp & \multicolumn{2}{c}{Up-front Exp} & \multicolumn{3}{c}{Critical Mazes Exp} & \multicolumn{2}{>{\centering}m{2.5cm}}{Process-Tracing Exp (Initial Mazes)} & \multicolumn{2}{>{\centering}m{2.5cm}}{Process-Tracing Exp (Critical Mazes)} \\" + \
    "\n" + r"\cmidrule(lr){2-2}\cmidrule(lr){3-4}\cmidrule(lr){5-7}\cmidrule(lr){8-9}\cmidrule(lr){10-11}",
)

global_lesion_string = global_lesion_string.replace(
    r"\bottomrule",
    r"\bottomrule"+"\n"+r"\multicolumn{10}{l}{$^*$Largest increase in AIC after lesioning; \textsuperscript{\textdagger}Second-largest increase}"
)

with open("results/global_lesion_table.tex", "w") as file:
    file.write(
        textwrap.dedent(r"""
        \documentclass{standalone}
        \usepackage{booktabs}
        \usepackage{tabularx}
        \usepackage{caption}
        \usepackage[scaled=0.7025996185887785]{helvet}
        \linespread{0.7025996185887785}
        \renewcommand{\familydefault}{\sfdefault}
        \begin{document}
        \fontsize{7}{8.4}
        """) + \
        global_lesion_string + \
        textwrap.dedent(r"""
        \end{document}
        """)
    )

# Create pdf to combine in vector image editor
# need to figure out how to make text in right font and also recognizable as text in eps
process = subprocess.Popen("xelatex -output-directory results results/global_lesion_table.tex".split())
output, error = process.communicate()
print(output, error)
global_lesion_table

This is XeTeX, Version 3.14159265-2.6-0.99992 (TeX Live 2015) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./results/global_lesion_table.tex
LaTeX2e <2015/01/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2012/09/15 v1.1b Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifluatex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/ifxetex/ifxetex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2015/

Initial Exp  Up-front Exp               Critical Mazes Exp  \
                 Awareness Awareness (F) Awareness (E)           Accuracy   
VGC             500.334119    280.542851    201.274375         112.984413   
Traj HS          -1.993593     -1.881579      1.874187          -1.794984   
Graph HS         30.705095      0.806911     -1.493090          -1.606933   
Bottleneck       69.683212     73.210511     96.774275          -1.834874   
SR Overlap       93.188111     55.174932     57.470380          -1.533044   
Nav Dist       1078.205811    508.168758           NaN           5.890297   
Nav Dist Step    12.188783     59.360752           NaN          -1.199853   
Opt Dist               NaN           NaN    248.677449                NaN   
Goal Dist        -0.281290     -1.638485     -1.283752           1.454585   
Start Dist       17.588037      3.660359      8.594189          -0.299094   
Wall Dist        94.161368     14.830999     24.247064          -1.976747   
Center Dist      90.495883     20.739777     29.697384          -1.834578   

                                      Process-Tracing Exp (Initial Mazes)  \
               Confidence   Awareness                            Hovering   
VGC            180.259432  487.345513                          293.947904   
Traj HS         -1.536673   -1.576486                           98.171705   
Graph HS         1.003789    1.489681                           18.392795   
Bottleneck       0.394212   13.747130                           48.160342   
SR Overlap      -1.945524    8.138670                           -1.603464   
Nav Dist        -0.546251   70.534111                                 NaN   
Nav Dist Step   -0.575870   40.485476                                 NaN   
Opt Dist              NaN         NaN                          171.108361   
Goal Dist        2.675984    1.559230                            8.806189   
Start Dist       7.493944   -1.917445                           59.260142   
Wall Dist        3.185226    0.762629                           19.589379   
Center Dist      9.043552   -1.983209                           33.763533   

                          Process-Tracing Exp (Critical Mazes)              
                 Duration                             Hovering    Duration  
VGC            176.337101                           184.626645  249.324479  
Traj HS         11.560456                            13.555646   13.304565  
Graph HS        12.699806                             5.230099    8.854590  
Bottleneck     112.117306                             2.014481    1.461123  
SR Overlap       0.346539                            96.638436   12.448785  
Nav Dist              NaN                                  NaN         NaN  
Nav Dist Step         NaN                                  NaN         NaN  
Opt Dist        -0.270734                            17.231649   20.781251  
Goal Dist       -0.849340                             8.977224   -1.742978  
Start Dist       2.455937                            36.898997   -1.225091  
Wall Dist       57.623653                            -1.659207    2.444861  
Center Dist     89.053794                            37.451778   36.915909

In [11]:
!xelatex -output-directory results results/global_lesion_table.tex

This is XeTeX, Version 3.14159265-2.6-0.99992 (TeX Live 2015) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./results/global_lesion_table.tex
LaTeX2e <2015/01/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2012/09/15 v1.1b Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifluatex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/ifxetex/ifxetex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2015/

In [12]:
annot_global_lesion_table.to_latex()

'\\begin{tabular}{lllllllllll}\n\\toprule\n{} &                       Initial Exp & \\multicolumn{2}{l}{Up-front Exp} & \\multicolumn{3}{l}{Critical Mazes Exp} & \\multicolumn{2}{l}{Process-Tracing Exp (Initial Mazes)} & \\multicolumn{2}{l}{Process-Tracing Exp (Critical Mazes)} \\\\\n{} &                         Awareness &                     Awareness (F) &                     Awareness (E) &                        Accuracy &                      Confidence &                        Awareness &                            Hovering &                          Duration &                             Hovering &                         Duration \\\\\n\\midrule\nVGC           &  500\\textbackslash textsuperscript\\{\\textbackslash textdagger\\} &  280\\textbackslash textsuperscript\\{\\textbackslash textdagger\\} &  201\\textbackslash textsuperscript\\{\\textbackslash textdagger\\} &                         112\\$\\textasciicircum *\\$ &                         180\\$\\textasciicircum *\\$ & 

# Static

In [13]:
predictors = [
    'static_vgc_weight_Z',
    'log_traj_based_hitcount_Z',
    'graph_based_hitcount_Z',
    'bottleneck_dist_Z',
    'sr_occ_Z',
    'nav_mindist_Z',
    'nav_mindist_timestep_Z',
    'optpolicy_dist_Z',
    'goal_dist_Z',
    'start_dist_Z',
    'walls_dist_Z',
    'center_dist_Z',
]
exp_analysis_predictors = dict(
    exp1=[p for p in predictors if "optpolicy_" not in p],
    exp2_early=[p for p in predictors if "nav_mindist" not in p],
    exp2_full=[p for p in predictors if "optpolicy_" not in p],
    exp3=[p for p in predictors if "optpolicy_" not in p],
    exp4=[p for p in predictors if "nav_mindist" not in p],
)
    


In [14]:
exp1_at = get_exp1_at()
exp1_at_im = ImmutableDataFrame(exp1_at)
exp1_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp1']),
    data=exp1_at_im
)
exp1_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 1 Accuracy",
    data=exp1_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp1'],
    rmods=rmods
)

exp2_at = get_exp2_at()
exp2_early_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'earlyterm'])
exp2_full_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'full'])
exp2_at_im = ImmutableDataFrame(exp2_at)
exp2_full_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp2_full']),
    data=exp2_full_at_im
)
exp2_early_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp2_early']),
    data=exp2_early_at_im
)
exp2_early_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 2 Accuracy Early",
    data=exp2_early_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp2_early'],
    rmods=rmods
)
exp2_full_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 2 Accuracy Full",
    data=exp2_full_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp2_full'],
    rmods=rmods
)


exp3_at, exp3_mt = get_exp3_at__exp3_mt()
exp3_at_im = ImmutableDataFrame(exp3_at)
exp3_mt_im = ImmutableDataFrame(exp3_mt)
exp3_correct_global_model = rmods.lmer(
    formula="correct ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_mt_im
)
exp3_conf_global_model = rmods.lmer(
    formula="conf_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_mt_im
)
exp3_attn_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_at_im
)
exp3_correct_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Accuracy",
    data=exp3_mt_im,
    dv="correct",
    model_func="glmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)
exp3_conf_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Confidence",
    data=exp3_mt_im,
    dv="conf_N",
    model_func="lmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)
exp3_attn_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Awareness",
    data=exp3_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)


exp4a_ht = get_exp4a_ht()
exp4b_ht = get_exp4b_ht()
exp4a_ht_im = ImmutableDataFrame(exp4a_ht)
exp4b_ht_im = ImmutableDataFrame(exp4b_ht)
exp4a_hov_global_model = rmods.glmer(
    formula="hovered ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4a_ht_im
)
exp4a_dur_global_model = rmods.lmer(
    formula="log_hoverduration ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4a_ht_im
)
exp4b_hov_global_model = rmods.glmer(
    formula="hovered ~ (1 | sessionId) + (1 | grid) +"+"+".join(exp_analysis_predictors['exp4']),
    data=exp4b_ht_im
)
exp4b_dur_global_model = rmods.lmer(
    formula="log_hoverduration ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4b_ht_im
)
exp4a_hov_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4a Hover",
    data=exp4a_ht_im,
    dv="hovered",
    model_func="glmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4a_dur_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4a Hover Duration",
    data=exp4a_ht_im,
    dv="log_hoverduration",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4b_hov_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4b Hover",
    data=exp4b_ht_im,
    dv="hovered",
    model_func="glmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4b_dur_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4b Hover Duration",
    data=exp4b_ht_im,
    dv="log_hoverduration",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:02<00:00,  5.40it/s]


## Create lesioned table

In [15]:
all_lesioning_res_static = [
    dict(res=exp1_lesioning_res, name=("Initial Exp", "Awareness")),
    dict(res=exp2_full_lesioning_res, name=("Up-front Exp", "Awareness (F)")),
    dict(res=exp2_early_lesioning_res, name=("Up-front Exp","Awareness (E)")),
    dict(res=exp3_correct_lesioning_res, name=("Critical Mazes Exp", "Accuracy")),
    dict(res=exp3_conf_lesioning_res, name=("Critical Mazes Exp", "Confidence")),
    dict(res=exp3_attn_lesioning_res, name=("Critical Mazes Exp", "Awareness")),
    dict(res=exp4a_hov_lesioning_res, name=("Process-Tracing Exp (Initial Mazes)", "Hovering")),
    dict(res=exp4a_dur_lesioning_res, name=("Process-Tracing Exp (Initial Mazes)", "Duration")),
    dict(res=exp4b_hov_lesioning_res, name=("Process-Tracing Exp (Critical Mazes)", "Hovering")),
    dict(res=exp4b_dur_lesioning_res, name=("Process-Tracing Exp (Critical Mazes)", "Duration")),
]

### Likelihood ratio test summary

In [16]:
all_summaries = []
for mod in all_lesioning_res_static:
    all_summaries.append(
        f"{', '.join(mod['name'])}: {mod['res'].likelihood_ratio_test('static_vgc_weight_Z', 1)}"
    )
all_summaries = '; \n'.join(all_summaries)
with open("./inputs/all_plan_exp_vgc_global_lesion_llr_svgc.tex", "w") as file:
    file.write(all_summaries)
print(all_summaries)

Initial Exp, Awareness: $\chi^2(1) = 400.63, p < 1.0 \times 10^{-16}$; 
Up-front Exp, Awareness (F): $\chi^2(1) = 203.67, p < 1.0 \times 10^{-16}$; 
Up-front Exp, Awareness (E): $\chi^2(1) = 125.41, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Accuracy: $\chi^2(1) = 86.07, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Confidence: $\chi^2(1) = 173.12, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Awareness: $\chi^2(1) = 376.52, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Initial Mazes), Hovering: $\chi^2(1) = 283.41, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Initial Mazes), Duration: $\chi^2(1) = 88.38, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Critical Mazes), Hovering: $\chi^2(1) = 210.25, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Critical Mazes), Duration: $\chi^2(1) = 241.61, p < 1.0 \times 10^{-16}$
